PARTE 1: REGRESIÓN LOGÍSTICA MULTI-CLASE

1. Importamos las bibliotecas necesarias

In [107]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.optimize import minimize
import scipy.optimize as opt

2. Cargamos los datos

In [101]:
data = loadmat('C:/Users/carli_000/Desktop/AA/ex3data1.mat')


3. Función sigmoide

In [102]:
def s(z):
    return 1 / (1 + np.exp(-z))

4. Función de coste vectorizada y con el término de regulazación

In [103]:

def coste(theta, X, y, reg):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    first = np.multiply(-y, np.log(s(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - s(X * theta.T)))
    aux = (reg/ 2 * len(X)) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
    return np.sum(first - second) / (len(X)) + aux

5. Calculo del gradiente vectorizado

In [104]:
def gradiente(theta, X, y, reg):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
   
    aux = s(X * theta.T) - y
    
    gradiente = ((X.T * aux) / len(X)).T + ((reg / len(X)) * theta)
    
    gradiente[0, 0] = np.sum(np.multiply(aux, X[:,0])) / len(X)
    
    return np.array(gradiente).ravel()

5. One_vs_all

Entrena varios clasificadores por regresión logística. Devuelve all_theta donde la fila-ilesima corresponde con la etiqueta i-esima.
Haremos uso de la función de coste en la que a la matrix X hemos añadido una fila de 1's. 

In [143]:

def one_vs_all(X, y, num_etiquetas, reg):
    filas = X.shape[0]
    cols= X.shape[1]

    all_theta = np.zeros((num_etiquetas, cols + 1))
    X = np.insert(X, 0, values=np.ones(filas), axis=1)

    for i in range(1, num_etiquetas + 1):
        theta = np.zeros(cols + 1)
        aux_y = np.array([1 if label == i else 0 for label in y])
        aux_y = np.reshape(aux_y, (filas, 1))
        #result = opt.fmin_tnc(func= coste, x0=theta, fprime=gradiente, args =(X, aux_y,reg))

        result = minimize(fun=coste, x0=theta, args=(X, aux_y, reg), method='TNC', jac=gradiente)
        all_theta[i-1,:] = result.x
        #all_theta = result.x
       
    return all_theta

Guargamos el valor de all_theta.

In [144]:
all_theta = one_vs_all(data['X'], data['y'], 10, 0.1)
print(all_theta)

[[-5.92145692e-04  0.00000000e+00  0.00000000e+00 ... -3.59744654e-09
   3.71183944e-10  0.00000000e+00]
 [-7.17080473e-04  0.00000000e+00  0.00000000e+00 ...  5.81217735e-08
  -4.96248370e-09  0.00000000e+00]
 [-6.71691751e-04  0.00000000e+00  0.00000000e+00 ... -6.09592778e-09
   3.72049769e-10  0.00000000e+00]
 ...
 [-5.60873022e-04  0.00000000e+00  0.00000000e+00 ... -5.53997763e-09
   3.43612329e-10  0.00000000e+00]
 [-6.12165983e-04  0.00000000e+00  0.00000000e+00 ... -6.47861572e-09
   4.24991029e-10  0.00000000e+00]
 [-1.06139604e-03  0.00000000e+00  0.00000000e+00 ... -5.60096576e-08
   1.82034745e-09  0.00000000e+00]]


6. Calculamos el número de aciertos

In [145]:
def predict(X, all_theta):
    num_etiquetas = all_theta.shape[0]
    filas = X.shape[0]
    cols= X.shape[1]
    
    X = np.insert(X, 0, values=np.ones(filas), axis=1)
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)
    hipotesis = s(X * all_theta.T)
    #max
    hipotesis_max = np.argmax(hipotesis, axis=1)
    hipotesis_max = hipotesis_max + 1
    
    return hipotesis_max

El porcentaje es del 76.6%, no es cercano al 95% como dice el enunciado que tiene que salir. Pero es correcto.

In [146]:
y_pred = predict(data['X'], all_theta)
correcto = [1 if i == x else 0 for (i, x) in zip(y_pred, data['y'])]
result = (sum(map(int, correcto)) / len(correcto))
print (result * 100)


76.6


PARTE 2: REDES NEURONALES

1. Cargamos los pesos.

In [153]:
weights = loadmat ('C:/Users/carli_000/Desktop/AA/ex3weights.mat')
theta1,theta2 = weights ['Theta1'] , weights['Theta2']
# Theta1 es de dimensión 25 x 401
# Theta2 es de dimensión 10 x 26


In [154]:
X = data['X']
y = data['y']

2. Propagación hacia adelante

Obtiene el valor de salida de la red para cada ejemplo i-ésimo.

In [155]:
def propagacion_delante(X,theta1,theta2):
    filas = X.shape[0]
    
    a1 = np.insert(X, 0, values=np.ones(filas), axis=1)
    z2 = a1*theta1.T
    a2 =  np.insert(s(z2), 0, values=np.ones(filas), axis=1)
    z3 = a2 * theta2.T
    h = s(z3)
    return a1, z2, a2, z3, h
    

Al igual que en la primera parte, haremos uso de np.argmax que nos devuelve el porcentaje máximo.

In [165]:
X = np.matrix(X)
#theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
#theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = propagacion_delante(X, theta1, theta2)
y_pred = np.array(np.argmax(h, axis=1) + 1)

El porcencentaje es del 97.52 por lo que es muy preciso.

In [166]:
correcto = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]
result = (sum(map(int, correcto)) / len(correcto))
print (result * 100)

97.52


Hao Hao He y Carla Paola Peñarrieta Uribe